In [1]:
from collections import deque, namedtuple
from IPython.core.debugger import set_trace
from IPython.display import display
import stringcase
import shutil
import sqlite3
import pandas as pd
import json
import os

In [2]:
CONFIG = 'last_save.json'
MAIN = 'philosopher.sparql'
BIRTH = 'birthPlace.sparql'
INFLUENCED = 'influenced.sparql'
MAIN_INTEREST = 'mainInterest.sparql'
NOTABLE_IDEA = 'notableIdea.sparql'
IDEA = 'notableIdea.sparql'
WORK = 'work.sparql'
SCHOOL = 'school.sparql'
ERA = 'era.sparql'

DB = './db.sqlite'
try:
    os.remove(os.path.join(os.getcwd(), DB))
except FileNotFoundError:
    pass

PARSE_FUNCS = {}

In [3]:
with open(CONFIG, 'r') as f:
    files = json.load(f)
files

{'work.sparql': '_data/work.sparql_Sun Mar 15 21:35:07 2020.json',
 'school.sparql': '_data/school.sparql_Sun Mar 15 21:35:10 2020.json',
 'philosopher.sparql': '_data/main_philosophers.sparql_Sun Mar 15 21:35:22 2020.json',
 'era.sparql': '_data/era.sparql_Sun Mar 15 21:35:42 2020.json',
 'birthPlace.sparql': '_data/birthPlace.sparql_Sun Mar 15 21:35:49 2020.json',
 'notableIdea.sparql': '_data/notableIdea.sparql_Sun Mar 15 21:35:59 2020.json',
 'mainInterest.sparql': '_data/mainInterest.sparql_Sun Mar 15 21:36:05 2020.json',
 'influenced.sparql': '_data/influenced.sparql_Sun Mar 15 21:36:26 2020.json'}

In [4]:
Sample = namedtuple('Sample', ['bindings', 'head'])

def load_sample(f, limit=None):
    data = json.load(f)
    if limit:
        return Sample(data["results"]["bindings"][:limit], data["head"]["vars"])
    return Sample(data["results"]["bindings"], data["head"]["vars"])

SAMPLES = {}

for file, name in files.items():
    with open(name, 'r') as f:
        SAMPLES[file] = load_sample(f, 50)

In [5]:
def parse_bindings(bindings, head, name):
    Result = namedtuple(name.split('.')[0], head, defaults=[None for key in head])
    res = deque()
    for binding in bindings:
        res.append(Result(**{key: data["value"] for key, data in binding.items()}))
    return res

def parse_sample(sample):
    return parse_bindings(sample.bindings, sample.head, type(sample).__name__)

sample_main = parse_sample(SAMPLES[MAIN])

In [6]:
def pick_not_none(data):
    def pick(c, n):
        if c is None:
            return n
        try:
            if len(n) > len(c):
                return n
        except:
            return c
    
    picked = [None for _ in data.columns]
    for datum in data.itertuples(index=False):
        picked = [
            pick(datum[i], picked[i])
            for i in range(len(picked))
        ]
        if all(val is not None for val in picked):
            return pd.Series(picked, index=data.columns)
    return pd.Series(picked, index=data.columns)

pick_not_none(pd.DataFrame([['a', None], [None, 'b']], columns=['q', 'w']))

q    a
w    b
dtype: object

In [7]:
def parse_object_name(object_name):
    return object_name.split('/')[-1].replace("_", ' ')

def parse_main(parsed):
    NAME_COLS = ['name', 'name2', 'birthName']
    ID = 'wikiPageID'
    NR_COLS = [ID, 'abstract', 'gender']

    df = pd.DataFrame(parsed)
    
    names = deque()
    for datum in df[[ID, *NAME_COLS]].itertuples(index=False):
        id_ = datum[0]
        [
            names.append((id_, name))
            for name in datum[1:]
            if name is not None and len(name) > 2
        ]
    df_names = pd.DataFrame(names, columns=[ID, 'name']).drop_duplicates() \
        .reset_index(drop=True).set_index([ID, 'name'])
    
    df_birthday = df[['wikiPageID', 'birthDate', 'deathDate']] \
        .groupby(['wikiPageID']).apply(pick_not_none) \
        .drop(ID, axis=1).dropna(how='all').reset_index().set_index('wikiPageID')
    df_nationality = df[['wikiPageID', 'nationality']].dropna().drop_duplicates()
    df_nationality.nationality = df_nationality.nationality.apply(parse_object_name)
    df_nationality = df_nationality.set_index(['wikiPageID', 'nationality'])
    df_phil = df[NR_COLS].drop_duplicates().set_index('wikiPageID').join(df_birthday)
    df_phil = df_phil.where(pd.notnull(df_phil), None)
    
    return {
        'PhilosopherHasName': df_names,
        'PhilosopherHasNationality': df_nationality,
        'Philosopher': df_phil
    }

PARSE_FUNCS[MAIN] = parse_main
[display(df.head(5)) for df in parse_main(sample_main).values()]

Empty DataFrame
Columns: []
Index: [(1731084, A. D. Gordon), (1731084, Aaron David Gordon), (2018, A. J. Ayer), (2018, Alfred Jules Ayer), (2018, Sir A. J. Ayer)]

,
wikiPageID,nationality
36243692,Indian
20990958,Indian


,abstract,gender,birthDate,deathDate
wikiPageID,,,,
1731084,Aaron David Gordon (Hebrew: אהרן דוד גורדון‎‎;...,male,1856-06-09,1922-02-22
2018,"Sir Alfred Jules ""Freddie"" Ayer (/ɛər/; 29 Oct...",male,1910-10-29,1989-06-27
36243692,"A Satyanarayana Shastri (June 2, 1925 – Januar...",male,1925-06-02,2004-01-14
41780272,Aaron Ben-Ze'ev (born 30 July 1949) is an Isra...,male,1949-07-30,None
2280663,Abas (Greek: Ἄβας) was an ancient Greek sophis...,male,None,None


[None, None, None]

In [8]:
def parse_birth_death(parsed):
    df = pd.DataFrame(parsed)
    ID = 'wikiPageID'
    BIRTH = 'birthPlace', 'birthPlace_wikiPageID'
    DEATH = 'deathPlace', 'deathPlace_wikiPageID'
    
    birth_df = df[[ID, *BIRTH]].drop_duplicates().dropna()
    death_df = df[[ID, *DEATH]].drop_duplicates().dropna()
    birth_df.birthPlace = birth_df.birthPlace.apply(parse_object_name)
    death_df.deathPlace = death_df.deathPlace.apply(parse_object_name)
    birth_df = birth_df.reset_index(drop=True).set_index(['wikiPageID', 'birthPlace'])
    death_df = death_df.reset_index(drop=True).set_index(['wikiPageID', 'deathPlace'])
    
    return {
        'PhilosopherWasBorn': birth_df,
        'PhilosopherDied': death_df
    }

PARSE_FUNCS[BIRTH] = parse_birth_death
[display(df.head()) for df in parse_birth_death(parse_sample(SAMPLES[BIRTH])).values()]

birthPlace_wikiPageID
wikiPageID birthPlace                          
1731084    Zhytomyr                     1025103
           Russian Empire              20611504
2018       England                         9316
           London                         17867
9008105    Kadhimiya                     695173

deathPlace_wikiPageID
wikiPageID deathPlace                                        
1731084    Degania Alef                               1601743
           British Mandate of Palestine              27276911
2018       England                                       9316
           London                                       17867
36243692   Karnataka                                    16880

[None, None]

In [9]:
def parse_influenced(parsed):
    df = pd.DataFrame(parsed)
    index = ['object1_wikiPageID', 'object2_wikiPageID']
    df = df[index].reset_index(drop=True).set_index(index)
    return {'PhilosopherInfluenced': df}

PARSE_FUNCS[INFLUENCED] = parse_influenced
[display(df.head()) for df in parse_influenced(parse_sample(SAMPLES[INFLUENCED])).values()]

Empty DataFrame
Columns: []
Index: [(39267803, 3894784), (1621559, 243451), (1731084, 2843640), (2018, 243824), (2018, 729989)]

[None]

In [10]:
def parse_stuff(parsed, name, attrs=None, key=None):
    if key is None:
        key = name
    if attrs is None:
        attrs = [f'{name}', f'{name}_abstract', f'{name}_wikiPageID']
    df = pd.DataFrame(parsed)
    df[name] = df[name].apply(parse_object_name)
    
    df_int = df[attrs].copy()
    df_int.columns = [attrs[0], *[attr.replace(name, "").replace('_', "") for attr in attrs[1:]]]
    df_int = df_int.drop_duplicates().reset_index(drop=True).set_index(key)
    
    df_has = df[['wikiPageID', key]]
    df_has.columns = 'philosopher_wikiPageID', key
    df_has = df_has.drop_duplicates().reset_index(drop=True).set_index(['philosopher_wikiPageID', key])
    
    return {
        stringcase.capitalcase(name): df_int,
        f'PhilopherHas{stringcase.capitalcase(name)}': df_has
    }

PARSE_FUNCS[MAIN_INTEREST] = lambda sample: parse_stuff(sample, 'mainInterest')
[
    display(df.head())
    for df in PARSE_FUNCS[MAIN_INTEREST](parse_sample(SAMPLES[MAIN_INTEREST])).values()
]

,abstract,wikiPageID
mainInterest,,
Jewish philosophy,Jewish philosophy (Hebrew: פילוסופיה יהודית‎‎)...,166200
Epistemology,Epistemology (/ᵻˌpɪstᵻˈmɒlədʒi/; from Greek ἐ...,9247
Ethics,Ethics or moral philosophy is a branch of phil...,9258
Philosophy of science,Philosophy of science is a branch of philosoph...,37010
Philosophy of language,Philosophy of language seeks to understand the...,6880370


Empty DataFrame
Columns: []
Index: [(1731084, Jewish philosophy), (1731084, Epistemology), (1731084, Ethics), (2018, Philosophy of science), (2018, Philosophy of language)]

[None, None]

In [11]:
PARSE_FUNCS[IDEA] = lambda sample: parse_stuff(sample, 'notableIdea')
[
    display(df.head())
    for df in PARSE_FUNCS[IDEA](parse_sample(SAMPLES[IDEA])).values()
]

,abstract,wikiPageID
notableIdea,,
Direct Experience vs Consciousness,None,None
Emotivist ethics,None,3760749
Verification principle,None,1205695
Pan-Islamism,None,None
Islamic Jurisprudence,None,12219121


Empty DataFrame
Columns: []
Index: [(1731084, Direct Experience  vs Consciousness), (2018, Emotivist ethics), (2018, Verification principle), (11340731, Pan-Islamism), (5108403, Islamic Jurisprudence)]

[None, None]

In [12]:
PARSE_FUNCS[WORK] = lambda sample: parse_stuff(
    sample, 'work', attrs = ['work', 'work_name', 'work_abstract', 'work_wikiPageID', 'work_language']
)

[
    display(df.head())
    for df in PARSE_FUNCS[WORK](parse_sample(SAMPLES[WORK])).values()
]

,name,abstract,wikiPageID,language
work,,,,
"Language, Truth, and Logic","Language, Truth, and Logic","Language, Truth, and Logic is a 1936 work of p...",3291133,English
The Wealth of Nations,The Wealth of Nations..,An Inquiry into the Nature and Causes of the W...,725991,English
The Theory of Moral Sentiments,The Theory of Moral Sentiments,The Theory of Moral Sentiments is a 1759 book ...,515983,None
The Century (book),The Century,"The Century is a book about politics, philosop...",16060654,None
On Being a Pagan,On Being a Pagan,"On Being a Pagan, originally published in Fren...",13355860,French


Empty DataFrame
Columns: []
Index: [(2018, Language, Truth, and Logic), (1814, The Wealth of Nations), (1814, The Theory of Moral Sentiments), (713688, The Century (book)), (652176, On Being a Pagan)]

[None, None]

In [13]:
PARSE_FUNCS[SCHOOL] = lambda sample: parse_stuff(sample, 'philosophicalSchool')
[
    display(df.head())
    for df in PARSE_FUNCS[SCHOOL](parse_sample(SAMPLES[SCHOOL])).values()
]

,abstract,wikiPageID
philosophicalSchool,,
Labor Zionism,Labor Zionism or Socialist Zionism (Hebrew: צִ...,41585591
Existential philosophy,None,126975
Analytic philosophy,Analytic philosophy (sometimes analytical phil...,159211
Logical positivism,"Logical positivism and logical empiricism, whi...",18403
Sophism,Sophism is a method of teaching. In ancient Gr...,23939752


Empty DataFrame
Columns: []
Index: [(1731084, Labor Zionism), (1731084, Existential philosophy), (2018, Analytic philosophy), (2018, Logical positivism), (2280663, Sophism)]

[None, None]

In [14]:
PARSE_FUNCS[ERA] = lambda sample: parse_stuff(sample, 'era')
[
    display(df.head())
    for df in PARSE_FUNCS[ERA](parse_sample(SAMPLES[ERA])).values()
]

,abstract,wikiPageID
era,,
20th-century philosophy,20th-century philosophy saw the development of...,806549
19th-century philosophy,In the 19th century the philosophies of the En...,209565
Contemporary philosophy,Contemporary philosophy is the present period ...,806560
Medieval philosophy,Medieval philosophy is the philosophy in the e...,26571896
Modern Era,None,351789


,
philosopher_wikiPageID,era
1731084,20th-century philosophy
2018,20th-century philosophy
11340731,19th-century philosophy
1254755,Contemporary philosophy
8242199,Medieval philosophy


[None, None]

In [15]:
def parse_all():
    data = {}
    for file, name in files.items():
        with open(name, 'r') as f:
            parsed = parse_sample(load_sample(f))
            data[file] = PARSE_FUNCS[file](parsed)
    return data

data = parse_all()

In [16]:
def save_df(df, table, conn):
    print('Saving:', table)
    df.to_sql(table, conn, if_exists='replace')

def make_db(data):
    try:
        conn = sqlite3.connect(DB)
        for output in data.values():
            for table, df in output.items():
                save_df(df, table, conn)
        conn.commit()
    finally:
        conn.close()
        
make_db(data)

Saving: Work
Saving: PhilopherHasWork
Saving: PhilosophicalSchool
Saving: PhilopherHasPhilosophicalSchool
Saving: PhilosopherHasName
Saving: PhilosopherHasNationality
Saving: Philosopher
Saving: Era
Saving: PhilopherHasEra
Saving: PhilosopherWasBorn
Saving: PhilosopherDied
Saving: NotableIdea
Saving: PhilopherHasNotableIdea
Saving: MainInterest
Saving: PhilopherHasMainInterest
Saving: PhilosopherInfluenced


In [17]:
def create_indices():
    INDICES = [
        ('')
    ]
    try:
        conn = sqlite3.connect(DB)
        c = conn.cursor()
        
    finally:
        conn.close()